In [1]:
import psycopg2
import json
import sys
import os
from shapely.geometry import MultiPolygon, asShape
from shapely.geometry.polygon import Polygon
import folium
import csv

regiao_ibge = "../../../../free_wheels/scripts/python/data/distritos-latlong.json"

f = open(regiao_ibge, 'r')
mapa = json.load(f)

regioes = {}
regioes_recebe_beneficio = {}
regioes_renda = {}
regioes_renda_recebe_beneficio = {}

for regiao in mapa['features']:
    regioes[regiao['properties']['ds_codigo']] = 0
    regioes_recebe_beneficio[regiao['properties']['ds_codigo']] = 0
    regioes_renda[regiao['properties']['ds_codigo']] = 0
    regioes_renda_recebe_beneficio[regiao['properties']['ds_codigo']] = 0

dbname = 'SMPED'
user = 'lima'
pswd = '123456'

try:
    conn = psycopg2.connect("dbname='"+dbname+"' user='"+user+"' host='localhost' password='"+pswd+"'")
except:
    print ("Nao foi possivel conectar com o banco")
    exit()

try:
    cur = conn.cursor()
    for regiao in mapa['features']:
        cur.execute("select count(*), AVG(VLR_RENDA_MEDIA_FAM) from dados_bpc where regiao_ibge = '"+str(regiao['properties']['ds_codigo'])+"'")      
        row = cur.fetchall()
        regioes[regiao['properties']['ds_codigo']] = int(row[0][0])
        if row[0][1] is not None:
            regioes_renda[regiao['properties']['ds_codigo']] = int(100*float(row[0][1]))/100
        
        cur.execute("select count(*), AVG(VLR_RENDA_MEDIA_FAM) from dados_bpc where RECEBE_BPC_RMV = 'SIM' and regiao_ibge = '"+str(regiao['properties']['ds_codigo'])+"'")
        row = cur.fetchall()
        regioes_recebe_beneficio[regiao['properties']['ds_codigo']] = int(row[0][0])   
        if row[0][1] is not None:
            regioes_renda_recebe_beneficio[regiao['properties']['ds_codigo']] = int(100*float(row[0][1]))/100
        
except:
    e = sys.exc_info()
    print(e)
    print ("Select falhou")
    exit()

conn.close()

def save_file(name, dic):
    with open(name,'w') as f:
#         w = csv.writer(sys.stderr)
        w = csv.writer(f)
        w.writerow(['ID', 'valor'])
        w.writerows(dic.items())

save_file('data/regioes.csv', regioes)
save_file('data/regioes_renda.csv', regioes)
save_file('data/regioes_recebe_beneficio.csv', regioes_recebe_beneficio)
save_file('data/regioes_renda_recebe_beneficio.csv', regioes_renda_recebe_beneficio)